In [243]:
# Dependencies and Setup
import pandas as pd
import numpy as np

#   * Total Schools[x] 
#   * Total Students[x]
#   * Total Budget[x]
#   * Average Math Score[x]
#   * Average Reading Score[x]
#   * % Passing Math[x]
#   * % Passing Reading[x]
#   * Overall Passing Rate (Average of the above two)[x]

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [244]:
school_data_complete.head(5)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [245]:
# Calculate the total reviews for the entire dataset
# total_reviews = goodreads_df["One Star Reviews"].sum() + goodreads_df["Two Star Reviews"].sum(
# ) + goodreads_df["Three Star Reviews"].sum() + goodreads_df["Four Star Reviews"].sum() + goodreads_df["Five Star Reviews"].sum()
# # Place all of the data found into a summary DataFrame
# summary_table = pd.DataFrame({"Total Unique Authors": author_count,
#                               "Earliest Year": [earliest_year],
#                               "Latest Year": [latest_year],
#                               "Total Reviews": [total_reviews]})
# author_count = len(goodreads_df["Authors"].unique())

total_schools = len(school_data_complete["school_name"].unique())
total_students = school_data_complete["Student ID"].count()
total_budget = school_data_complete["budget"].unique().sum()
avg_math = school_data_complete["math_score"].mean()
avg_read = school_data_complete["reading_score"].mean()
overall_pass = (avg_math + avg_read)/2
passing_math = ((school_data_complete["math_score"] >= 70).sum()/total_students) * 100
passing_read = ((school_data_complete["reading_score"] >= 70).sum()/total_students) * 100
#TODO: create new summary table[x]

In [246]:
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                                 "Total Students": [total_students],
                                 "Total Budget": [total_budget],
                                 "Average Math Score": [avg_math],
                                 "Average Reading Score": [avg_read],
                                 "% Passing Math": [passing_math],
                                 "% Passing Reading": [passing_read]})
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463


In [247]:
# * Create an overview table that summarizes key metrics about each school, including:
#  you may need a group by on school name
#   * School Name[x]
#   * School Type[x]
#   * Total Students[x]
#   * Total School Budget[x]
#   * Per Student Budget[x]
#   * Average Math Score[x]
#   * Average Reading Score[x]
#   * % Passing Math[x]
#   * % Passing Reading[x]
#   * Overall Passing Rate (Average of the above two)[x]

school_data_complete["Passing Math Students"] = np.where(school_data_complete["math_score"] >= 70, 1, 0)
school_data_complete["Passing Reading Students"] = np.where(school_data_complete["reading_score"] >= 70, 1, 0)
# group on school name (will create index) and aggregate columns into new dataframe
school_group_df = school_data_complete.groupby(["school_name"]).agg({"student_name": "count","budget":"mean","math_score":"mean","reading_score":"mean","Passing Math Students":"sum","Passing Reading Students":"sum","size":"mean"})

school_group_df["Per Student Budget"] = school_group_df["budget"]/school_group_df["student_name"]
school_group_df["% Passing Math"] = (school_group_df["Passing Math Students"]/school_group_df["student_name"]) * 100
school_group_df["% Passing Reading"] = (school_group_df["Passing Reading Students"]/school_group_df["student_name"]) * 100
school_group_df["% Overall Passing"] = (school_group_df["% Passing Math"] + school_group_df["% Passing Reading"])/2

# merge new grouped dataframe with original school data to bring school tpye back
merged_df = pd.merge(school_group_df,school_data, on="school_name", how="inner")

school_group_df = merged_df.rename(columns={"student_name":"Total Students","budget_x":"Total School Budget","type":"School Type","math_score":"Average Math Score","reading_score":"Average Reading Score","type":"School Type"})

#reduce the final output of columns in new dataframe, but keep school_group_df because it is used further down
#drop size_y, budget_y, budget_y
#make school_name the index for the final dataframe
school_summary = school_group_df.drop(columns=["size_x","size_y","budget_y","Passing Math Students","Passing Reading Students","School ID"])
school_summary = school_summary[["school_name","School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
school_summary = school_summary.set_index("school_name")
# remove school name header
school_summary.index.name = None
school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


In [248]:
# Top Performing Schools (By Passing Rate)
# Sort and display the top five schools in overall passing rate

top_performing = school_summary.sort_values("% Overall Passing", ascending=False)
top_performing.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [249]:
# Bottom Performing Schools (By Passing Rate)
# Sort and display the five worst-performing schools

bottom_performing = school_summary.sort_values("% Overall Passing", ascending=True)
bottom_performing.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [250]:
# Math Scores by Grade
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
# Create a pandas series for each grade. Hint: use a conditional statement.
# Group each series by school
# Combine the series into a dataframe
# Optional: give the displayed data cleaner formatting

math_score_grades = pd.DataFrame({"school name":school_data_complete["school_name"],
                                  "grade":school_data_complete["grade"],
                                  "math score":school_data_complete["math_score"]})
                                                  
# use where fuction to build numerator and denominator                                 
math_score_grades["9th"] = np.where(school_data_complete["grade"]=='9th',school_data_complete["math_score"],0) #sum this
math_score_grades["9th graders"] = np.where(school_data_complete["grade"]=='9th',1,0) #denominator for 9th grade averages
math_score_grades["10th"] = np.where(school_data_complete["grade"]=='10th',school_data_complete["math_score"],0) #sum this
math_score_grades["10th graders"] = np.where(school_data_complete["grade"]=='10th',1,0) #denominator for 9th grade averages
math_score_grades["11th"] = np.where(school_data_complete["grade"]=='11th',school_data_complete["math_score"],0) #sum this
math_score_grades["11th graders"] = np.where(school_data_complete["grade"]=='11th',1,0) #denominator for 9th grade averages
math_score_grades["12th"] = np.where(school_data_complete["grade"]=='12th',school_data_complete["math_score"],0) #sum this
math_score_grades["12th graders"] = np.where(school_data_complete["grade"]=='12th',1,0) #denominator for 9th grade averages
new_df = math_score_grades.groupby(["school name"]).agg({"9th":"sum","9th graders":"sum","10th":"sum","10th graders":"sum","11th":"sum","11th graders":"sum","12th":"sum","12th graders":"sum"})

# Do math
new_df["avg 9th grade math"] = new_df["9th"]/new_df["9th graders"]
new_df["avg 10th grade math"] = new_df["10th"]/new_df["10th graders"]
new_df["avg 11th grade math"] = new_df["11th"]/new_df["11th graders"]
new_df["avg 12th grade math"] = new_df["12th"]/new_df["12th graders"]

# Drop support columns
final_math_df = new_df.drop(columns=["9th","9th graders","10th","10th graders","11th","11th graders","12th","12th graders"])
final_math_df = final_math_df.rename(columns={"avg 9th grade math":"9th","avg 10th grade math":"10th","avg 11th grade math":"11th","avg 12th grade math":"12th"})
final_math_df.index.name = None
# average math scores by grade
final_math_df

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


In [251]:
read_score_grades = pd.DataFrame({"school name":school_data_complete["school_name"],
                                  "grade":school_data_complete["grade"],
                                  "reading score":school_data_complete["reading_score"]})

# reading score grades                           
read_score_grades["9th"] = np.where(school_data_complete["grade"]=='9th',school_data_complete["reading_score"],0) #sum this
read_score_grades["9th graders"] = np.where(school_data_complete["grade"]=='9th',1,0) #denominator for 9th grade averages
read_score_grades["10th"] = np.where(school_data_complete["grade"]=='10th',school_data_complete["reading_score"],0) #sum this
read_score_grades["10th graders"] = np.where(school_data_complete["grade"]=='10th',1,0) #denominator for 9th grade averages
read_score_grades["11th"] = np.where(school_data_complete["grade"]=='11th',school_data_complete["reading_score"],0) #sum this
read_score_grades["11th graders"] = np.where(school_data_complete["grade"]=='11th',1,0) #denominator for 9th grade averages
read_score_grades["12th"] = np.where(school_data_complete["grade"]=='12th',school_data_complete["reading_score"],0) #sum this
read_score_grades["12th graders"] = np.where(school_data_complete["grade"]=='12th',1,0) #denominator for 9th grade averages
new_df = read_score_grades.groupby(["school name"]).agg({"9th":"sum","9th graders":"sum","10th":"sum","10th graders":"sum","11th":"sum","11th graders":"sum","12th":"sum","12th graders":"sum"})


new_df["avg 9th grade read"] = new_df["9th"]/new_df["9th graders"]
new_df["avg 10th grade read"] = new_df["10th"]/new_df["10th graders"]
new_df["avg 11th grade read"] = new_df["11th"]/new_df["11th graders"]
new_df["avg 12th grade read"] = new_df["12th"]/new_df["12th graders"]

final_read_df = new_df.drop(columns=["9th","9th graders","10th","10th graders","11th","11th graders","12th","12th graders"])
final_read_df = final_read_df.rename(columns={"avg 9th grade read":"9th","avg 10th grade read":"10th","avg 11th grade read":"11th","avg 12th grade read":"12th"})
final_read_df.index.name = None
# average reading scores by grade
final_read_df

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


In [252]:
# Scores by School Spending
# Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
# Average Math Score
# Average Reading Score
# % Passing Math
# % Passing Reading
# Overall Passing Rate (Average of the above two)

bins_df = pd.DataFrame({"Per Student Budget": school_group_df["Per Student Budget"],
                        "Average Math Score": school_group_df["Average Math Score"],
                        "Average Reading Score": school_group_df["Average Reading Score"],
                        "% Passing Math": school_group_df["% Passing Math"],
                        "% Passing Reading": school_group_df["% Passing Reading"],
                        "% Overall Passing Rate": school_group_df["% Overall Passing"]})

# # Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# Make columns on the fly using bins
bins_df["Spending Ranges Per Student"] = pd.cut(bins_df["Per Student Budget"], spending_bins, labels=group_names)

bins_df = bins_df.groupby(["Spending Ranges Per Student"]).agg({"Average Math Score":"mean","Average Reading Score":"mean","% Passing Math":"mean","% Passing Reading":"mean","% Overall Passing Rate":"mean"})

bins_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges Per Student,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [253]:
# Scores by School Size
# Perform the same operations as above, based on school size.
# Create a table that breaks down school performances based on school size. Use 4 reasonable bins to group school size. Include in the table each of the following:
# Average Math Score
# Average Reading Score
# % Passing Math
# % Passing Reading
# Overall Passing Rate (Average of the above two)

size_bins_df = pd.DataFrame({"School Size": school_group_df["size_x"],
                        "Average Math Score": school_group_df["Average Math Score"],
                        "Average Reading Score": school_group_df["Average Reading Score"],
                        "% Passing Math": school_group_df["% Passing Math"],
                        "% Passing Reading": school_group_df["% Passing Reading"],
                        "% Overall Passing Rate": school_group_df["% Overall Passing"]})

# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Make columns on the fly using bins
size_bins_df["Scores by School Size"] = pd.cut(size_bins_df["School Size"], size_bins, labels=group_names)
size_bins_df = size_bins_df.groupby(["Scores by School Size"]).agg({"Average Math Score":"mean","Average Reading Score":"mean","% Passing Math":"mean","% Passing Reading":"mean","% Overall Passing Rate":"mean"})
size_bins_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Scores by School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [254]:
# Scores by School Type
# Perform the same operations as above, based on school type.
# Create a table that breaks down school performances based on school size. Use 4 reasonable bins to group school size. Include in the table each of the following:
# Average Math Score
# Average Reading Score
# % Passing Math
# % Passing Reading
# Overall Passing Rate (Average of the above two)

# no need to specify bins because school type is included in data
type_bins_df = school_group_df.groupby(["School Type"]).agg({"Average Math Score":"mean","Average Reading Score":"mean","% Passing Math":"mean","% Passing Reading":"mean","% Overall Passing":"mean"})

type_bins_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [ ]:
# Have a nice day :] 